In [66]:
import pandas as pd
import numpy as np
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib
import re
from datetime import datetime

In [ ]:
!pip3 install streamlit

In [68]:

import streamlit as st
import os  # For file checking

In [ ]:
print("Loading JSON file...")
with open('memes.json', 'r') as file:
    data = pd.read_json(file)

print("Converting '_default' to DataFrame...")
memes_df = pd.DataFrame(data['_default'].values)
if len(memes_df.columns) == 1 and isinstance(memes_df.iloc[0, 0], dict):
    memes_df = pd.DataFrame(list(memes_df[0]))

In [ ]:
#load the data into a dataframe
memes_df = memes_df[['title', 'ups', 'created_utc']].dropna()
print("Initial DataFrame:")
print(memes_df.head())

In [ ]:
#set the threshold for virality as the 75th percentile of upvotes (top 25% are considered viral)
threshold = memes_df['ups'].quantile(0.75)  
print(f"Threshold for virality: {threshold} upvotes")
memes_df['is_viral'] = (memes_df['ups'] >= threshold).astype(int)
memes_df = memes_df.drop(columns=['ups'])

In [ ]:
#count the number of words and characters in the title
memes_df['word_count'] = memes_df['title'].apply(lambda x: len(str(x).split()))
memes_df['char_count'] = memes_df['title'].apply(len)
memes_df['sentiment'] = memes_df['title'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
print("DataFrame with Features:")
print(memes_df.head())

In [73]:
#count the number of emojis in the text using unicode ranges
def count_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # Emoticons
                               u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # Transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # Flags
                               u"\U00002700-\U000027BF"  # Dingbats
                               u"\U000024C2-\U0001F251"  # Enclosed characters 
                               "]+", flags=re.UNICODE)
    return len(emoji_pattern.findall(str(text)))

memes_df['emoji_count'] = memes_df['title'].apply(count_emojis)

In [ ]:
# Extract posting hour from UTC timestamp for timing analysis
def get_hour(timestamp):
    return datetime.utcfromtimestamp(int(timestamp)).hour

memes_df['post_hour'] = memes_df['created_utc'].apply(get_hour)

In [75]:
# Count uppercase letters in the title for emphasis analysis
def count_caps(text):
    return sum(1 for char in str(text) if char.isupper())

memes_df['cap_count'] = memes_df['title'].apply(count_caps)

In [ ]:
#print the dataframe with all features
print("DataFrame with all features:")
print(memes_df.head())


#split the data into training and testing sets

X = memes_df[['word_count', 'char_count', 'sentiment', 'emoji_count', 'post_hour', 'cap_count']]
y = memes_df['is_viral']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Train Random Forest Classifier on the scaled feature data
model = RandomForestClassifier(random_state=42)
model.fit(X_train_scaled, y_train)

In [ ]:
#test the model
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
#save the model and scaler
joblib.dump(model, 'meme_predictor_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
print("Model and scaler saved!")

In [80]:
#predict the virality of a meme
def predict_meme_virality(title, post_hour=None):
    word_count = len(title.split())
    char_count = len(title)
    sentiment = TextBlob(title).sentiment.polarity
    emoji_count = count_emojis(title)
    cap_count = count_caps(title)
    post_hour = post_hour if post_hour is not None else 12  
    
    new_meme = pd.DataFrame({
        'word_count': [word_count],
        'char_count': [char_count],
        'sentiment': [sentiment],
        'emoji_count': [emoji_count],
        'post_hour': [post_hour],
        'cap_count': [cap_count]
    })
    new_meme_scaled = scaler.transform(new_meme)
    prediction = model.predict(new_meme_scaled)[0]
    prob = model.predict_proba(new_meme_scaled)[0][1]
    return "Viral" if prediction == 1 else "Not Viral", prob



In [81]:
# Function to load or train the model 
def load_or_train_model():
    model_file = 'meme_predictor_model.pkl'
    scaler_file = 'scaler.pkl'
    
    if os.path.exists(model_file) and os.path.exists(scaler_file):
        model = joblib.load(model_file)
        scaler = joblib.load(scaler_file)
        print("Loaded pre-trained model and scaler.")  
    else:
        print("Loading JSON file...")
        with open('memes.json', 'r') as file:
            data = pd.read_json(file)
        
        print("Converting '_default' to DataFrame...")
        memes_df = pd.DataFrame(data['_default'].values)
        if len(memes_df.columns) == 1 and isinstance(memes_df.iloc[0, 0], dict):
            memes_df = pd.DataFrame(list(memes_df[0]))
        
        memes_df = memes_df[['title', 'ups', 'created_utc']].dropna()
        print("Initial DataFrame:")
        print(memes_df.head())
        
        threshold = memes_df['ups'].quantile(0.75)
        print(f"Threshold for virality: {threshold} upvotes")
        memes_df['is_viral'] = (memes_df['ups'] >= threshold).astype(int)
        memes_df = memes_df.drop(columns=['ups'])
        
        memes_df['word_count'] = memes_df['title'].apply(lambda x: len(str(x).split()))
        memes_df['char_count'] = memes_df['title'].apply(len)
        memes_df['sentiment'] = memes_df['title'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
        memes_df['emoji_count'] = memes_df['title'].apply(count_emojis)
        memes_df['post_hour'] = memes_df['created_utc'].apply(get_hour)
        memes_df['cap_count'] = memes_df['title'].apply(count_caps)
        
        print("DataFrame with all features:")
        print(memes_df.head())
        
        X = memes_df[['word_count', 'char_count', 'sentiment', 'emoji_count', 'post_hour', 'cap_count']]
        y = memes_df['is_viral']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        model = RandomForestClassifier(random_state=42)
        model.fit(X_train_scaled, y_train)
        
        y_pred = model.predict(X_test_scaled)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Model Accuracy: {accuracy:.2f}")
        print("Classification Report:")
        print(classification_report(y_test, y_pred))
        
        joblib.dump(model, model_file)
        joblib.dump(scaler, scaler_file)
        print("Model and scaler saved!")
    
    return model, scaler

In [82]:
#predict the virality of a meme
def predict_meme_virality(title, post_hour=None, model=None, scaler=None):
    word_count = len(title.split())
    char_count = len(title)
    sentiment = TextBlob(title).sentiment.polarity
    emoji_count = count_emojis(title)
    cap_count = count_caps(title)
    post_hour = post_hour if post_hour is not None else 12  
    
    new_meme = pd.DataFrame({
        'word_count': [word_count],
        'char_count': [char_count],
        'sentiment': [sentiment],
        'emoji_count': [emoji_count],
        'post_hour': [post_hour],
        'cap_count': [cap_count]
    })
    new_meme_scaled = scaler.transform(new_meme)
    prediction = model.predict(new_meme_scaled)[0]
    prob = model.predict_proba(new_meme_scaled)[0][1]
    
    result = "Viral" if prediction == 1 else "Not Viral"
    suggestions = []
    
    # If not viral, provide suggestions based on feature analysis
    if prediction == 0:
        if word_count < 5:
            suggestions.append("Try making the title longer (5+ words) for more context or humor.")
        if emoji_count == 0:
            suggestions.append("Add some emojis (😄👍) to make it more engaging!")
        if cap_count < 2:
            suggestions.append("Use more CAPS for emphasis (e.g., 'THIS IS FUNNY').")
        if sentiment <= 0:
            suggestions.append("Make it more positive or humorous for better appeal.")
        if post_hour not in range(17, 22):  
            suggestions.append("Post between 5 PM and 10 PM for higher engagement.")
        if suggestions:
            st.write("**Suggestions to improve virality:**")
            for suggestion in suggestions:
                st.write(f"- {suggestion}")  
    
    return result, prob, suggestions

In [ ]:
#Streamlit app 
def main():
    st.title("Meme Virality Predictor")  
    st.write("Enter a meme and posting hour to predict if it’ll go viral!")  
    
    # Load or train model
    model, scaler = load_or_train_model()
    
    # Streamlit: User input fields
    title = st.text_area("")  
    post_hour = st.slider("Posting Hour (0-23)", 0, 23, 12) 
    
   
    if st.button("Predict"):  # Button to trigger prediction
        result, prob, suggestions = predict_meme_virality(title, post_hour, model, scaler)
        st.write(f"**Prediction**: {result}")  
        st.write(f"**Probability of being viral**: {prob:.2%}") 

if __name__ == "__main__":
    main()